In [59]:
import h5py, sys
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import folium

# hf.close()
# with h5py.File("BayouContraband.g07.hdf",'r') as hf:
hf = h5py.File("BayouContraband.g07.hdf",'r')

In [60]:
hf['/Geometry/Cross Sections/Polyline Info']
df = pd.DataFrame(hf['/Geometry/Cross Sections/Polyline Info'])

In [61]:

# df = pd.DataFrame(hf['/Geometry/Cross Sections/Polyline Info'])
# df
# 0=part_count_accumulated
# 1=part_count
# 2=part_index
# 3=part_type
df.rename(columns={0:'part_count_accumulated',1:'part_count',2:'part_index',3:'part_type'}, inplace=True)
# df

In [62]:
df_elev = pd.DataFrame(hf["Geometry/Cross Sections/Station Elevation Info"])
df_elev.rename(columns={0: "point_count_accum", 1: "xs_n_points"}, inplace=True)
# df_elev

In [63]:
df = pd.concat([df, df_elev], axis=1, join='inner')
df

,part_count_accumulated,part_count,part_index,part_type,point_count_accum,xs_n_points
0,0,3,0,1,0,65
1,3,3,1,1,65,65
2,6,4,2,1,130,78
3,10,7,3,1,208,70
4,17,7,4,1,278,76
...,...,...,...,...,...,...
103,606,5,103,1,22790,275
104,611,4,104,1,23065,352
105,615,4,105,1,23417,209
106,619,4,106,1,23626,138


In [64]:
# hf["Geometry/Cross Sections/Station Elevation Info"])
points = hf["Geometry/Cross Sections/Station Elevation Info"][0,1]
points

65

In [65]:
coord_index_start_points = 0
for i,v in enumerate(df['part_index']):
    points = hf["Geometry/Cross Sections/Station Elevation Info"][i,1]
    # print (i, parts)
    coord_index_end_points = coord_index_start_points + points
    coords_range = [coord_index_start_points, coord_index_end_points]
    bounds_points = [hf['/Geometry/Cross Sections/Station Elevation Values'][coord_index_start_points:coord_index_end_points].tolist()][0]

    # get min value from bounds_points
    min_value = min(bounds_points)
    
    # update start of range for next XS in loop.
    coord_index_start_points += points

In [66]:
gdf = gpd.GeoDataFrame()
m = folium.Map([30.2, -90.3], zoom_start=8, tiles='cartodbdark_matter')

In [ ]:
coord_index_start = 0
coord_index_start_points = 0

for i,v in enumerate(df['part_index']):
    parts = hf['/Geometry/Cross Sections/Polyline Parts'][i,1]
    # print (i, parts)
    coord_index_end = coord_index_start + parts
    coords_range = [coord_index_start, coord_index_end]
    bounds = [hf['/Geometry/Cross Sections/Polyline Points'][coord_index_start:coord_index_end].tolist()][0]
    # update start of range for next XS in loop.
    coord_index_start += parts

    points = hf["Geometry/Cross Sections/Station Elevation Info"][i,1]
    # print (i, parts)
    coord_index_end_points = coord_index_start_points + points
    coords_range = [coord_index_start_points, coord_index_end_points]
    bounds_points = [hf['/Geometry/Cross Sections/Station Elevation Values'][coord_index_start_points:coord_index_end_points].tolist()][0]
    # get min value from bounds_points
    min_value = min(bounds_points)
    
    # update start of range for next XS in loop.
    coord_index_start_points += points

    # print (coords_range)
    line_geom = LineString(bounds)
    line = gpd.GeoDataFrame(index=[i], geometry=[line_geom], data={'name': v, 'min_elev': min_value[1], 'min_sta': min_value[0]})
    # line['name'] = [v.decode("utf-8")]
    # line['name'] = [str(v)]
    # print(str(v))
    # line['style'] = [style]
    # line.crs = 'PROJCS["NAD_1983_TCMS_Albers_FtUS",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",4921250.0],PARAMETER["False_Northing",19685000.0],PARAMETER["Central_Meridian",-100.0],PARAMETER["Standard_Parallel_1",27.5],PARAMETER["Standard_Parallel_2",35.0],PARAMETER["Latitude_Of_Origin",18.0],UNIT["Foot_US",0.3048006096012192]]'
    line.crs = 'EPSG:3452'
    gdf = gdf.append(line)

In [68]:
# gdf

In [69]:
hf.close()

In [70]:
gdf = gdf.to_crs(epsg=4326)
g = folium.GeoJson(data=gdf).add_to(m)
folium.GeoJsonTooltip(fields=["name", "min_elev"]).add_to(g)


m

In [71]:
gdf

,name,min_elev,min_sta,geometry
0,0,20.52,3980.989990,"LINESTRING (-93.25753 30.16288, -93.25712 30.1..."
1,1,20.52,3980.989990,"LINESTRING (-93.25753 30.16288, -93.25712 30.1..."
2,2,20.75,3976.290039,"LINESTRING (-93.25738 30.16295, -93.25703 30.1..."
3,3,20.84,3993.419922,"LINESTRING (-93.25736 30.16307, -93.25689 30.1..."
4,4,20.93,4010.560059,"LINESTRING (-93.25728 30.16308, -93.25660 30.1..."
...,...,...,...,...
103,103,13.55,2140.139893,"LINESTRING (-93.24834 30.18303, -93.24282 30.1..."
104,104,12.71,1770.729980,"LINESTRING (-93.24837 30.18330, -93.23907 30.1..."
105,105,12.37,3632.620117,"LINESTRING (-93.24159 30.18495, -93.24047 30.1..."
106,106,12.47,3529.800049,"LINESTRING (-93.24154 30.18497, -93.23922 30.1..."


In [ ]:
gdf.to_file("output.geojson", driver="GeoJSON")